https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

https://github.com/pytorch/examples/blob/main/word_language_model/model.py

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import random
import pandas as pd

In [2]:
from transformer import TransformerModel

In [14]:
from net import Net_CBOW
import numpy as np
version = "april27_WT2_nodatalim_10epoch_128dim_100minf"

# vocab = torch.load(f"saves/vocab_{version}.pt")
len(vocab)

6908

In [119]:
torch.save(vocab, "saves/vocab_may1_WT2_transformer_min25f.pt")

In [5]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-2-v1")

DATA_SPLIT = "train"
text = wikitext2[DATA_SPLIT]['text']
text = [item.lower().strip() for item in text if len(item) > 0]
text = [item.split(" ") + ["\n"] for item in text if "=" not in item]
# Prepare Corpus
DATA_LIMIT = None #paragraph limit
all_words = []
for paragraph in text[:DATA_LIMIT]:
    all_words += paragraph
all_words = pd.Series(all_words)
len(all_words)
len(all_words.unique())

v_counts = all_words.value_counts()

#filter out rare words
N_times = 25 #shows up in 100 different paragraphs
# v_counts = all_words.value_counts()
corpus = pd.Series([key for key in v_counts.keys() if v_counts[key] >= N_times])
corpus
vocab = {}
for i in range(len(corpus)):
    vocab[corpus[i]] = i
len(vocab)
#6908

6908

In [6]:
vocab = torch.load("saves/vocab_may1_WT2_transformer_min25f.pt")

In [9]:
def lookup_id(word, vocab=vocab):
    if word not in vocab:
        return vocab["<unk>"]
    return vocab[word]
def lookup_token(word_id, vocab=vocab):
    for word in vocab:
        if vocab[word] == word_id:
            return word
    return "<unk>"

In [7]:
def split_list(l, item, include_item = False):
    particular_value = item
    result = []
    temp_list = []
    for i in l:
        if i == particular_value:
            if include_item:
                temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    return result
split_list(["5", "6", "7", "8", "9", "8", "9", "a", "b"], "c")

[['5', '6', '7', '8', '9', '8', '9', 'a', 'b']]

In [4]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-103-v1")
text_train = wikitext2["train"]['text']
text_train = [item.lower().strip() for item in text_train if len(item) > 0]
text_test = wikitext2["test"]['text']
text_test = [item.lower().strip() for item in text_test if len(item) > 0]
# len(text_test)
text_train = [item.split(" ") + ["\n"] for item in text_train if "=" not in item]
text_test = [item.split(" ") + ["\n"] for item in text_test if "=" not in item]

/Users/gregengel/Documents/GitHub/Simple-Chatbot/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gregengel/Documents/GitHub/Simple-Chatbot/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1644653.57 examples/s]


In [10]:

max_seq_length = 128
buffer = 0

# x_train = [[lookup_id(word) for word in split_list(paragraph, ".", True)[1]] for paragraph in text_train if len(split_list(paragraph, ".", True))>=3]
# y_train = [item[1:] for item in x_train]
# x_train = [item[:-1] for item in x_train]

# x_test = [[lookup_id(word) for word in split_list(paragraph, ".", True)[1]] for paragraph in text_test if len(split_list(paragraph, ".", True))>=3]
# print(len(x_test))
# y_test = [item[1:] for item in x_test]
# x_test = [item[:-1] for item in x_test]
def seq_length():
    return random.randint(5, max_seq_length)

x_train = [[lookup_id(word) for word in paragraph[:max_seq_length+1]] for paragraph in text_train if len(paragraph) >= max_seq_length + buffer+1]
y_train = [item[1:] for item in x_train]
x_train = [item[:-1] for item in x_train]
print("-->")
x_test = [[lookup_id(word) for word in paragraph[:seq_length()+1]] for paragraph in text_test if len(paragraph) >= max_seq_length + buffer+1]
y_test = [item[1:] for item in x_test]
x_test = [item[:-1] for item in x_test]
len(x_test[0]), len(x_test), len(x_train)

-->


(30, 761, 342274)

In [14]:
print([lookup_token(i) for i in x_train[6871]])
print([lookup_token(i) for i in y_train[6871]])

['<unk>', 'had', 'a', 'pet', 'common', 'starling', 'which', 'could', 'sing', 'part', 'of', 'his', 'piano', '<unk>', 'in', 'g', 'major', '(', '<unk>', '.', '<unk>', ')', '.', 'he', 'had', 'bought', 'it', 'from', 'a', 'shop', 'after', 'hearing', 'it', 'sing', 'a', 'phrase', 'from', 'a', 'work', 'he', 'wrote', 'six', 'weeks', 'previously', ',', 'which', 'had', 'not', 'yet', 'been', 'performed', 'in', 'public', '.', 'he', 'became', 'very', 'attached', 'to', 'the', 'bird', 'and', 'arranged', 'an', 'elaborate', 'funeral', 'for', 'it', 'when', 'it', 'died', 'three', 'years', 'later', '.', 'it', 'has', 'been', 'suggested', 'that', 'his', 'a', 'musical', '<unk>', '(', 'k.', '<unk>', ')', 'might', 'be', 'written', 'in', 'the', '<unk>', ',', '<unk>', 'style', 'of', 'a', 'starling', "'s", '<unk>', '.', 'other', 'people', 'who', 'have', 'owned', 'common', 'starlings', 'report', 'how', '<unk>', 'they', 'are', 'at', '<unk>', 'up', '<unk>', 'and', '<unk>', '.', 'the', 'words', 'have', 'no', 'meaning',

In [15]:
x_train = x_train[6872:]
y_train = y_train[6872:]
x_test = x_test[:200]
y_test = y_test[:200]


In [18]:
len(x_test[2])

119

In [19]:
len(x_train[0])

128

In [20]:
from transformer import TransformerModel

In [21]:
v_size = len(vocab)
d_model = 512
num_heads = 8
num_layers = 6
d_feedfoward = 2048
dropout = 0.5

# transformer = TransformerModel(ntoken = v_size, ninp = d_model, nhead = num_heads, nhid = d_feedfoward, nlayers = num_layers, dropout = dropout)

print(len(x_train))
x_train = x_train
y_train = y_train
# src_data = torch.tensor(x_train)

# unk_id = lookup_id("<unk>")
# count_unk = sum(sum((i == unk_id) for i in paragraph) for paragraph in x_train).item()
# count_total = sum(sum(1 for i in paragraph) for paragraph in x_train)
# print(count_unk, count_total, count_unk/count_total)

# a = torch.randint(1, src_vocab_size, (64, max_seq_length))
# print(a[:, 1:])
# print(a[:, :-1])
len(x_test)


335402


200

In [28]:
net_file = "saves/model_transformer_may3_1200am.pt"
transformer = torch.load("saves/model_transformer_may2_0600pm.pt")

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.00002, betas=(0.9, 0.98), eps=1e-9) #normal lr is 0.0001

transformer.train()

batch_size = 40
num_batches = len(x_train) // batch_size
BATCH_PRINT_SIZE = 10
print("Batches per epoch:", num_batches/BATCH_PRINT_SIZE)

percent_data_per_epoch = 1

indices = list(range(len(x_train)))
for epoch in range(3):
    train_loss = 0
    x_train_copy = [x_train[indices[i]] for i in range(len(indices))]
    y_train_copy = [y_train[indices[i]] for i in range(len(indices))]
    for batch in range(int(num_batches * percent_data_per_epoch)):
        source = torch.tensor(x_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        target = torch.tensor(y_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        # print(source)
        # print(target)
        optimizer.zero_grad()
        output = transformer(source)
        output = output.view(-1, v_size)
        loss = criterion(output, target.view(-1))
        # print(output.shape)
        # print(target.view(-1).shape)
        # print("________")
        # print(loss)
        loss.backward()
        optimizer.step()
        if batch % BATCH_PRINT_SIZE == 0:
            print("|", end="")
        train_loss += loss.item()
    print(" -> ", end='')
    #eval
    test_loss = 0
    count_loss = 0
    with torch.no_grad():
        for datax, datay in zip(x_test,y_test):
            source = torch.tensor([datax])
            target = torch.tensor([datay]).view(-1)
            output = transformer(source)
            output = output.view(-1, v_size)
            test_loss += criterion(output, target).item() * len(datax)
            count_loss += len(datax)
    test_loss /= count_loss

    train_loss /= int(num_batches * percent_data_per_epoch)
    random.shuffle(indices)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}")
torch.save(transformer, net_file)

Batches per epoch: 838.5
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| -> Epoch: 1, Train Loss: 5.060751532769018, Test Loss: 4.998311252736597
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [161]:
# net_file = "saves/model_transformer_may1_1250pm.pt"
# torch.save(transformer, net_file)

In [162]:
o = transformer(src_data[0:1])
sm = np.array(torch.softmax(o, 1)[0].detach())
ids = [list(v).index(max(v)) for v in sm]
words = [lookup_token(i) for i in ids]
print([lookup_token(i) for i in src_data[0]])
print(words)

['card', '.', 'the', '<unk>', 'were', 'performed', 'on', 'the', 'fastra', 'ii', ',', 'the', 'fastra', 'i', ',', 'a', '<unk>', '@-@', 'core', '<unk>', '(', 'consisting', 'of', '<unk>', '<unk>', ')', ',', 'an', '<unk>', '<unk>', '<unk>', '<unk>', 'card', 'on', 'an', '<unk>', 'core', '<unk>', '<unk>', '<unk>', ',', 'and', 'on', 'an', '<unk>', 'core', '<unk>', '<unk>', '<unk>', 'itself', '.', 'the', 'fastra', 'ii', 'is', 'over', 'three', 'times', 'faster', 'than', 'the', 'fastra', 'i', 'in', '<unk>', '<unk>', 'reconstruction', 'speed', '.', 'although', 'the', 'fastra', 'ii', '<unk>', 'more', 'power', 'than', 'the', 'fastra', 'i', ',', 'it', "'s", 'nearly', '3', 'times', 'as', 'energy', '<unk>', 'as', 'the', 'fastra', 'i', ',', 'and', 'over', '300', 'times', 'as', 'energy', '<unk>', 'as', 'the', '<unk>', '@-@', 'core', '<unk>', '.', 'the', 'video', 'cards', 'run', 'at', '37', 'degrees', '<unk>', 'when', '<unk>', ',', 'and', 'at', '60', 'degrees', '<unk>', 'at', 'full', 'load', '.']
['with',

In [47]:
text = "you are a helpful assistant . Answer the following question . "

text = [lookup_id(word.lower()) for word in text.strip().split(" ")]

for count in range(50):
    i = torch.tensor([text])
    o = transformer(i)
    sm = np.array(torch.softmax(o, 2)[0].detach())
    top5 = [np.zeros(5) for _ in range(len(sm))]
    top5p = [np.zeros(5) for _ in range(len(sm))]
    for vi in range(len(sm)-1,len(sm)):
        v = sm[vi]
        for item in v:
            m = top5[vi][list(top5[vi]).index(min(top5[vi]))]
            if lookup_token(list(v).index(item)) != "<unk>":
                top5[vi][list(top5[vi]).index(min(top5[vi]))] = max(m, item)
        top5[vi] = [list(v).index(i) for i in top5[vi]]
        top5p[vi] = [v[i] for i in top5[vi]]
    # ids = [list(v).index(max(v)) for v in sm]
    words = [[lookup_token(i) for i in w] for w in top5][-1]
    # print([lookup_token(i) for i in src_data[0]])
    # print(words)
    print(' '.join([lookup_token(i) for i in text]))
    print(words)
    print(top5p[-1])
    chosen_word = words[int(input("Choose a word: "))-1]
    text.append(lookup_id(chosen_word))
# text


you are a <unk> assistant . answer the following question .
['in', 'a', 'he', '"', 'on']
[0.082577735, 0.024254575, 0.043053746, 0.033439506, 0.025467215]
you are a <unk> assistant . answer the following question . in
['a', 'their', 'an', 'his', 'which']
[0.06289544, 0.013856521, 0.015006466, 0.020354144, 0.0133204125]
you are a <unk> assistant . answer the following question . in their
['opponents', 'first', 'way', 'home', 'own']
[0.008090098, 0.070655584, 0.011046601, 0.0079570925, 0.059320744]
you are a <unk> assistant . answer the following question . in their own
['right', ',', '.', '@-@', 'and']
[0.011739017, 0.0643455, 0.044036668, 0.00968784, 0.025577338]
you are a <unk> assistant . answer the following question . in their own right
['@-@', ',', '.', 'to', 'and']
[0.09095287, 0.09226749, 0.05907558, 0.110956185, 0.04225587]
you are a <unk> assistant . answer the following question . in their own right ,
['in', 'a', 'which', 'but', 'and']
[0.021232849, 0.025981423, 0.031597324, 

ValueError: invalid literal for int() with base 10: ''

In [ ]:
device = torch.device("cpu")
ntokens = len(vocab)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
model = transformer
temperature = 2
log_interval = 1
input = 'alongside the main story <unk>'
input = [lookup_id(i) for i in input.strip().split(" ")]
input = torch.tensor(input).view(len(input), 1)


with open('out_generation.txt', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(20):
            output = model(input, False)
            word_weights = output[-1].squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            word_tensor = torch.Tensor([[word_idx]]).long().to(device)
            print(word_tensor)
            input = torch.cat([input, word_tensor], 0)

            word = lookup_token(word_idx)

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % log_interval == 0:
                print('| Generated {}/{} words'.format(i, [lookup_token(i[0])for i in input]))

tensor([[1220]])
| Generated 0/['alongside', 'the', 'main', 'story', '<unk>', 'terms'] words
tensor([[1074]])
| Generated 1/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location'] words
tensor([[4]])
| Generated 2/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>'] words
tensor([[4]])
| Generated 3/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>'] words
tensor([[489]])
| Generated 4/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon'] words
tensor([[1216]])
| Generated 5/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish'] words
tensor([[180]])
| Generated 6/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish', 'you'] words
tensor([[1251]])
| Generated 7/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish', 'you', 'individ